<img src='../../../common/DH_LOGO.jpeg' align='left' width=100%/>

In [1]:
# initial setup
#%run "../../../common/0_notebooks_base_setup.py"

# Agregação.

# Pandas 2 - Split, Apply, Combine

## Intro

Separar um conjunto de dados em categorias e aplicar uma função a cada grupo, que pode ser agregação, transformação ou filtro, é uma etapa muito comum em um fluxo de trabalho de análise de dados.

Depois de carregar e preparar um conjunto de dados, podemos precisar calcular estatísticas de grupo ou, possivelmente, tabelas dinâmicas para gerar relatórios ou visualizações.

`pandas` fornece métodos que nos permitem realizar essas tarefas naturalmente.

Nestes guias, aprenderemos a

* Dividir um objeto `pandas` em partes usando uma ou mais keys

* Calcular medidas de resumo em grupos, como quantidade, média, desvio padrão ou qualquer função definida pelo usuário

* Aplicar transformações por grupos.

* Construir tabelas dinâmicas

## Dataset

O [GCBA](https://www.buenosaires.gob.ar/) realiza pesquisas com turistas que vêm aos centros de atendimento. Pergunta-se o motivo da consulta, os dias da viagem, o país de origem, entre outros.

O conjunto de dados está publicamente acessível no portal de dados abertos do [GCBA](https://data.buenosaires.gob.ar/dataset/encuesta-centros-atencion-turistica-cat).

Neste guia usaremos o dataset Resultado de pesquisas em Centros de Atenção ao Turista (CAT) em 2017-2018.

## Problema

A partir dos dados das consultas aos postos de turismo da cidade de Buenos Aires, vamos responder a perguntas sobre o país de origem dos turistas, algumas medidas estatísticas sobre o número de dias de permanência na cidade e o número de visitantes.

## GroupBy

Podemos descrever as operações em grupos com o termo *split-apply-combine*.

Na primeira etapa do processo, os dados em um objeto `pandas` (uma instância de `Series` ou de `DataFrame`) se dividem em grupos (*split*) com base em uma ou mais keys que definimos. Esta divisão é feita por linhas (axis = 0) ou por colunas (axis = 1).

Como um segundo estágio, aplicamos uma função a cada um dos grupos (*apply*) resultando em um novo valor por grupo.

Na última etapa, os resultados da aplicação da função em cada um dos grupos são combinados em um objeto de resultado (*combine*).


As chaves que agrupamos podem ser especificadas de várias maneiras diferentes:

* Uma lista ou numpy array do mesmo tamanho que o eixo selecionado

* Para objetos DataFrame, uma string que indica o nome da coluna pela qual vamos agrupar.

* Para objetos DataFrame, uma string que indica o nome do índice pelo qual vamos agrupar.

* Um dicionário ou Series que estabelece um mapeamento entre um valor e o nome do grupo.

* Uma função python que será avaliada em cada um dos rótulos dos eixos.

* Uma lista com qualquer uma das opções acima.


Observe que o resultado de cada uma dessas opções é **produzir uma matriz de valores que usaremos para dividir** o objeto Series ou DataFrame.


---

Vamos ler os dados da pesquisa de turismo GCBA, ver o quão grande é o DataFrame, quais colunas ele tem, que tipo de dados ele é e os primeiros registros lidos.

In [3]:
import pandas as pd
import numpy as np

In [4]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).

data = pd.read_csv("../AgregationPivotTable/resultado-de-encuestas-2017-2018.csv", sep = ",", low_memory = False) 

print(data.shape)
print(data.columns)
print(data.dtypes)

data.head()

(174812, 20)
Index(['id', 'fecha', 'centro_atencion_turistica', 'barrio', 'comuna',
       'pasajeros', 'pais_residencia_si_extranjero',
       'otro_pais_residencia_si_extranjero',
       'provincia_residencia_si_argentino', 'pernoctaciones',
       'medio_transporte_llegada', 'alojamiento', 'otro_alojamiento',
       'barrio.1', 'otro_barrio', 'primera_vez', 'motivo_viaje',
       'otro_motivo_viaje', 'motivo_consulta', 'otro_motivo_consulta'],
      dtype='object')
id                                      int64
fecha                                  object
centro_atencion_turistica              object
barrio                                 object
comuna                                 object
pasajeros                               int64
pais_residencia_si_extranjero          object
otro_pais_residencia_si_extranjero     object
provincia_residencia_si_argentino      object
pernoctaciones                        float64
medio_transporte_llegada               object
alojamiento          

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
0,1,2017-01-02,retiro,RETIRO,COMUNA 1,2,Chile,NaN,NaN,7.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alojamiento,NaN
1,2,2017-01-02,retiro,RETIRO,COMUNA 1,1,NaN,NaN,Provincia de Buenos Aires,2.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion,NaN
2,3,2017-01-02,retiro,RETIRO,COMUNA 1,2,NaN,NaN,Córdoba,0.0,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alrededores,NaN


Vamos ver qual porcentagem de registros nulos em cada coluna:

In [4]:
data.isnull().sum() / data.shape[0]*100

id                                     0.000000
fecha                                  0.000000
centro_atencion_turistica              0.002860
barrio                                 0.000000
comuna                                 0.000000
pasajeros                              0.000000
pais_residencia_si_extranjero         42.734480
otro_pais_residencia_si_extranjero    98.274146
provincia_residencia_si_argentino     58.163627
pernoctaciones                        49.878155
medio_transporte_llegada               0.000000
alojamiento                           41.654463
otro_alojamiento                      99.943368
barrio.1                              93.905453
otro_barrio                           99.846120
primera_vez                            0.000000
motivo_viaje                           7.813537
otro_motivo_viaje                     99.915338
motivo_consulta                        0.000000
otro_motivo_consulta                  99.228886
dtype: float64

Vemos que existem várias colunas com uma porcentagem muito alta de valores nulos.

## Agregações simples

### Pergunta 1:

A coluna pernoctaciones é do tipo numérica (float64), vamos calcular quantos dias no total e em média dura a viagem das pessoas que vieram a estes centros de atendimento.

In [5]:
pernoites_serie = data.pernoctaciones
print("Duração média da viagem:", pernoites_serie.mean())
print("Duração total da viagem:", pernoites_serie.sum())

Duração média da viagem: 7.422305664296557
Duração total da viagem: 650335.0


### Pergunta 2:

**2.a** Quantos e quais são os motivos da consulta?

Vamos analisar o campo `motivo_consulta` que não possui valores nulos.

Ajuda: [`pandas.Series.value_counts()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

**2.b** Quais são os 5 motivos mais consultados?

Ajuda: Vamos indexar a Series ordenada que é o resultado de value_counts.

(value_counts retorna a Series ordenada).

In [6]:
# Normalmente features numéricas avaliamos com "describe" e as categóricas avaliamos com "value_counts"
motivos = data.motivo_consulta.value_counts()
motivos.head()

motivo_consulta
Mapas Orientacion        22875
Orientacion              20235
Mapas                    11908
Bus Mapas Orientacion     8543
Bus                       7034
Name: count, dtype: int64

In [7]:
data['motivo_consulta'].value_counts()

motivo_consulta
Mapas Orientacion                                                             22875
Orientacion                                                                   20235
Mapas                                                                         11908
Bus Mapas Orientacion                                                          8543
Bus                                                                            7034
                                                                              ...  
Bus Mapas atractivos barrios que_hacer tango nocturna                             1
Bus Mapas Orientacion alrededores barrios gastro museos que_hacer nocturna        1
Mapas Orientacion agenda_cultural apps ferias museos parques surf visitas         1
Mapas Orientacion apps atractivos surf visitas                                    1
Mapas Orientacion compras gastro museos ni├▒os                                    1
Name: count, Length: 8451, dtype: int64

In [8]:
motivos_top5 = motivos[0 : 5]
motivos_top5

motivo_consulta
Mapas Orientacion        22875
Orientacion              20235
Mapas                    11908
Bus Mapas Orientacion     8543
Bus                       7034
Name: count, dtype: int64

### Pergunta 3:

Usando o método [`pandas.DataFrame.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), vamos avaliar as colunas pasajeros (passageiros) e pernoctaciones (pernoites).

Este método retorna um DataFrame, usando esse resultado vamos responder qual é o número médio de passageiros.

In [9]:
medidas = data[["pasajeros", "pernoctaciones"]].describe().round(2)
#print(medidas)
print(type(medidas))
medidas

<class 'pandas.core.frame.DataFrame'>


,pasajeros,pernoctaciones
count,174812.00,87619.00
mean,2.06,7.42
std,1.55,27.31
min,1.00,0.00
25%,1.00,3.00
50%,2.00,4.00
75%,2.00,5.00
max,150.00,690.00


In [10]:
print("Número médio de passageiros: ", medidas.loc["mean", "pasajeros"].round(2))
medidas

Número médio de passageiros:  2.06


,pasajeros,pernoctaciones
count,174812.00,87619.00
mean,2.06,7.42
std,1.55,27.31
min,1.00,0.00
25%,1.00,3.00
50%,2.00,4.00
75%,2.00,5.00
max,150.00,690.00


Vemos que o número máximo de dormidas é de 690 e o número máximo de passageiros é de 150.

Vamos ver quais registros têm esses valores. E vamos tentar entender se correspondem ou não a um erro.

In [11]:
data_passageiros_150_mask = data.pasajeros == 150
data_passageiros_150 = data.loc[data_passageiros_150_mask, ]
data_passageiros_150

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
19806,19808,2017-03-04,recoleta,RECOLETA,COMUNA 2,150,Francia,NaN,NaN,NaN,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,estudiante_extranj,NaN


Parece ser um contingente de estudantes franceses em trânsito ("Não pernoite em Buenos Aires").

In [12]:
data_pernoites_690_mask = data.pernoctaciones == 690
data_pernoites_690 = data.loc[data_pernoites_690_mask, ]
data_pernoites_690

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
84622,84633,2017-10-24,hub2,SAN NICOLAS,COMUNA 1,1,Colombia,NaN,NaN,690.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,instalaciones,NaN


Desse cadastro não podemos extrair informações que justifiquem 690 dias de permanência.


### DataFrameGroupBy


A definição abstrata de agrupamento é fornecer um mapeamento entre valores e (rótulos ou) nomes de grupos.

Um objeto [`pandas.DataFrame.groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) não calcula nada, mas cria uma estrutura intermediária com todas as informações necessárias para posteriormente aplicar alguma operação a cada grupo.

O **resultado dessa operação** é retornado em um Series ou DataFrame **indexado pelos valores exclusivos da key groupby**.

As operações que podemos aplicar em um objeto `groupby` estão listadas [aqui](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) Usando o resultado de uma operação de agrupamento por nome de coluna como exemplo, vamos apresentar algumas propriedades e métodos deste objeto.

Vamos agrupar os dados por coluna em `pais_residencia_si_extranjero`.

In [13]:
data_grouped = data.groupby('pais_residencia_si_extranjero')

#### Tipo 

Vamos ver que tipo é o objeto retornado:


In [14]:
type(data_grouped)

pandas.core.groupby.generic.DataFrameGroupBy

#### size

Vamos ver o número de registros em cada grupo e quantos registros do `DataFrame` original são atribuídos a algum grupo com o atributo [`.size()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.size.html).

In [15]:
data_grouped.size()

pais_residencia_si_extranjero
Alemania                                     3564
Australia                                    2548
Bolivia                                       902
Brasil                                      29979
Bélgica                                       304
Canadá                                       2219
Chile                                        9140
China                                         907
Colombia                                     5965
Corea del Sur                                 155
Costa Rica                                    421
Ecuador                                      1360
España                                       6249
Estados Unidos                               6826
Francia                                      5159
India                                         105
Israel                                        855
Italia                                       2373
Japón                                         801
Malasia             

In [16]:
data_grouped.size().sum()

100107

In [17]:
data_grouped.sum()

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
pais_residencia_si_extranjero,,,,,,,,,,,,,,,,,,,
Alemania,299725486,2017-01-022017-01-022017-01-022017-01-022017-0...,florida2florida2hub2recoleta2florida2florida2f...,SAN NICOLASSAN NICOLASSAN NICOLASRECOLETASAN N...,COMUNA 1COMUNA 1COMUNA 1COMUNA 2COMUNA 1COMUNA...,7265,0,0,20309.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,CruceroCruceroCruceroCruceroCruceroBarcoPor el...,PalermoSan TelmoMontserratConstituciónSan Nico...,FloresTigre,No especificaNo especificaNo especificaNo espe...,No especificaNo especificaNo especificaNo espe...,DE PASO,bicicletassubeMapas atractivos literario museo...,TigreHerramientas para arreglar biciWifiEstadi...
Australia,218388532,2017-01-022017-01-022017-01-022017-01-022017-0...,puertomadero2florida2recoleta2recoleta2florida...,PUERTO MADEROSAN NICOLASRECOLETARECOLETASAN NI...,COMUNA 1COMUNA 1COMUNA 2COMUNA 2COMUNA 1COMUNA...,5394,0,0,8786.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,CruceroCruceroCruceroCruceroCruceroCrucero,RecoletaPalermoSan NicolásRecoletaPalermoRecol...,5,No especificaNo especificaNo especificaNo espe...,No especificaNo especificaNo especificaNo espe...,Combinaci├│n de vuelos. Pasajero en tr├ínsito,MapasBusBusBusOrientacionBusOrientacionOrienta...,Correo argentinoPiletas públicasDay saving lig...
Bolivia,74537777,2017-01-022017-01-022017-01-022017-01-022017-0...,retiropuertomadero2florida2hub2florida2puertom...,RETIROPUERTO MADEROSAN NICOLASSAN NICOLASSAN N...,COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA...,1981,0,0,7948.0,No especificaAviónNo especificaNo especificaNo...,HotelNo pernocta en Buenos AiresNo pernocta en...,0,San NicolásMontserratConstituciónSan NicolásBa...,FloresVilla crespoLiniers5Liniers,No especificaSíNo especificaNo especificaNo es...,No especificaNo especificaNo especificaNo espe...,0,Mapas Orientacion organismosMapasBus MapasMapa...,TemaikenBA playaConsulta impresión voucher vis...
Brasil,2669210899,2017-01-022017-01-022017-01-022017-01-022017-0...,hub2florida2recoleta2florida2caminitocaminitof...,SAN NICOLASSAN NICOLASRECOLETASAN NICOLASBOCAB...,COMUNA 1COMUNA 1COMUNA 2COMUNA 1COMUNA 4COMUNA...,68984,0,0,101399.0,No especificaAviónAviónNo especificaNo especif...,No pernocta en Buenos AiresHostelNo pernocta e...,CruceroCruceroCruceroestan de paso aeropuertoB...,San TelmoPalermoSan NicolásRecoletaRetiroRetir...,BoedoSan cristobalNuñezSan isidroNo sabeVilla ...,No especificaSíNo especificaNo especificaNo es...,No especificaVacaciones / Ocio / Recreación No...,escalaEscalaTr├ínsitoTr├ínsitoTr├ínsitoNegocio...,Mapas Orientacion apps visitasBusBusOrientacio...,Pronóstico del tiempopaseo en barcoSan Antonio...
Bélgica,39699810,2017-10-202017-10-212017-10-212017-10-232017-1...,hub2pzasanmartinfloridahub2caminitorecoletacam...,SAN NICOLASRETIROSAN NICOLASSAN NICOLASBOCAREC...,COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA 4COMUNA...,664,0,0,1717.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,0,RecoletaBelgranoRetiroSan NicolásRetiroSan Tel...,0,No especificaNo especificaNo especificaNo espe...,No especificaNo especificaNo especificaNo espe...,0,Mapas OrientacionBus Mapas OrientacionMapas su...,Ayuda para instalar las app de ba turismoWifi ...
Canadá,181570557,2017-01-022017-01-022017-01-022017-01-022017-0...,florida2retiroflorida2florida2florida2florida2...,SAN NICOLASRETIROSAN NICOLASSAN NICOLASSAN NIC...,COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA...,4636,0,0,9249.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,CruceroCruceroCrucerovive en argCrucero,

In [18]:
data_grouped.size().sum() / data.shape[0]

0.5726551952955174

Apenas $52\%$ dos registros foram atribuídos a um grupo.

Vamos ver se esse número de registros atribuídos a um grupo corresponde ao número de registros não nulos nesse campo.

In [19]:
data.loc[data.pais_residencia_si_extranjero.notnull(), ].shape[0]

100107

#### Índices

É um dicionário cujas chaves são os valores únicos das chaves groupby, neste caso os valores da coluna `country_residence_if_extranjero`, e cujos valores são uma matriz com os [índices](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.indices.html#pandas.core.groupby.GroupBy.indices) do DataFrame cujo valor nessa coluna é igual ao da chave.

In [20]:
print(type(data_grouped.indices))
data_grouped.indices

<class 'dict'>


{'Alemania': array([    48,    108,    147, ..., 174778, 174794, 174802], dtype=int64),
 'Australia': array([    33,    242,    267, ..., 174421, 174496, 174774], dtype=int64),
 'Bolivia': array([    36,    150,    183,    228,    352,    420,    435,    483,
           506,   1090,   1092,   1850,   1880,   2051,   2193,   2204,
          2562,   2626,   2845,   3085,   3104,   3387,   3537,   4021,
          4408,   4437,   4541,   4564,   4667,   4873,   5164,   5237,
          5249,   5260,   5359,   5389,   5464,   5661,   5678,   6585,
          6684,   6774,   6856,   6863,   6878,   6945,   7251,   7310,
          7492,   7755,   7837,   8582,   8749,   8851,   8914,   8929,
          8949,   8964,   9263,   9531,   9542,   9978,  10531,  10829,
         11616,  11807,  11984,  12030,  12209,  12348,  12685,  12820,
         13007,  13356,  13615,  13660,  13789,  13790,  13791,  13792,
         13820,  14155,  14300,  14474,  14805,  15207,  15317,  15726,
         15863,  163

In [21]:
data_grouped.indices['Alemania']

array([    48,    108,    147, ..., 174778, 174794, 174802], dtype=int64)

Neste exemplo, os índices 48, 108, 147, ... correspondem à Alemanha; os índices 33, 242, 267, ... correspondem à Austrália, os índices 6, 8, 28, ... ao Uruguai. Vamos verificar isso:

In [22]:
data.pais_residencia_si_extranjero.loc[[48, 108, 147]]

48     Alemania
108    Alemania
147    Alemania
Name: pais_residencia_si_extranjero, dtype: object

In [23]:
data.pais_residencia_si_extranjero.loc[[33, 242, 267]]

33     Australia
242    Australia
267    Australia
Name: pais_residencia_si_extranjero, dtype: object

In [24]:
data.pais_residencia_si_extranjero.loc[[ 6, 8, 28]]

6     Uruguay
8     Uruguay
28    Uruguay
Name: pais_residencia_si_extranjero, dtype: object

#### groups

É semelhante a `index`, mas associando as chaves do groupby a objetos do tipo Index.

In [25]:
data_grouped.groups

{'Alemania': [48, 108, 147, 278, 291, 320, 334, 452, 456, 561, 576, 578, 647, 706, 726, 764, 795, 840, 856, 865, 913, 931, 978, 981, 1003, 1117, 1156, 1272, 1289, 1308, 1355, 1357, 1421, 1462, 1505, 1556, 1582, 1666, 1675, 1689, 1708, 1714, 1716, 1795, 1856, 1858, 1890, 1932, 1956, 2035, 2065, 2098, 2114, 2143, 2151, 2181, 2200, 2312, 2338, 2345, 2399, 2427, 2442, 2475, 2504, 2581, 2672, 2681, 2687, 2813, 2856, 2862, 2865, 2931, 2947, 2949, 3041, 3064, 3102, 3252, 3324, 3330, 3339, 3362, 3446, 3533, 3546, 3653, 3657, 3681, 3690, 3699, 3803, 3834, 3855, 3865, 3906, 3922, 4004, 4089, ...], 'Australia': [33, 242, 267, 328, 337, 340, 373, 688, 889, 1060, 1067, 1091, 1413, 1445, 1502, 1535, 1589, 1672, 1679, 1871, 1878, 1920, 2019, 2020, 2167, 2184, 2258, 2330, 2358, 2386, 2396, 2467, 2530, 2544, 2572, 2652, 2663, 2783, 2867, 2874, 2927, 3043, 3151, 3189, 3199, 3204, 3221, 3241, 3304, 3435, 3501, 3541, 3581, 3591, 3660, 3663, 3684, 3716, 3731, 3745, 3875, 4037, 4039, 4150, 4204, 4213, 4241,

### Groupby com dicionários e Series como chave

Até agora vimos como definir uma ou mais colunas de um DataFrame como a key do groupby. Quando fazemos isso, estamos definindo para cada registro um rótulo (o valor do campo ou campos-chave nesse registro) que usaremos para determinar a qual grupo ele pertence.

A seguir, veremos exemplos de Series e dicionários como chaves do groupby.

Para agrupar usando uma instância de Series ou dicionário como chave, precisamos que os valores do índice da Series ou das chaves do dicionário sejam os mesmos do índice do DataFrame no qual queremos grupo.

Vamos definir um dicionário que associa os países ao seu contêiner:

In [26]:
data['pais_residencia_si_extranjero'].value_counts()

pais_residencia_si_extranjero
Brasil                                      29979
Chile                                        9140
Estados Unidos                               6826
España                                       6249
Colombia                                     5965
Francia                                      5159
Uruguay                                      3986
Alemania                                     3564
México                                       3252
Otro país de residencia si es extranjero     3017
Reino Unido                                  2701
Australia                                    2548
Italia                                       2373
Canadá                                       2219
Venezuela                                    2157
Perú                                         2031
Ecuador                                      1360
China                                         907
Bolivia                                       902
Paraguay            

In [27]:
data['pais_residencia_si_extranjero'].value_counts().shape

(36,)

In [28]:
# Nesse caso, dá pra fazer um group by com uma coisa que é externa. 
# Como por exemplo, a informação do continente que é algo que não se tinha antes;
pais_em_continente = {
    'Chile': 'America', 'Francia': 'Europa', 'México': 'America', 'Colombia': 'America', 
    'Uruguay': 'America', 'Estados Unidos': 'America', 'España': 'Europa', 'Italia': 'Europa', 
    'India': 'Asia', 'Ecuador': 'America', 'Brasil': 'America',
    'Australia': 'Oceania', 'Bolivia': 'America', 'Reino Unido': 'Europa', 
    'Alemania': 'Europa', 'Israel': 'Asia', 'China': 'Asia', 
    'Venezuela': 'America', 'Países Bajos': 'Europa', 'Canadá': 'America', 'Suiza': 'Europa', 'Turquía': 'Europa',
    'Noruega': 'Europa', 'Corea del Sur': 'Asia', 'Polonia': 'Europa', 'Perú': 'America', 'Paraguay': 'America',
    'Costa Rica': 'America', 'Japón': 'Asia', 'Marruecos': 'Africa', 'Bélgica': 'Europa', 'Malasia': 'Asia', 
    'Rusia': 'Europa', 'Sudáfrica': 'Africa', 'Nueva Zelanda': 'Oceania'        
}

Vamos atribuir como índice do DataFrame os valores do campo pais_residencia_si_extranjero, que coincidem com as chaves do dicionário que definimos acima.

In [29]:
# Eu to colocando essa feature como indíce do meu dataset
data.index = data.pais_residencia_si_extranjero

In [30]:
data

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
pais_residencia_si_extranjero,,,,,,,,,,,,,,,,,,,,
Chile,1,2017-01-02,retiro,RETIRO,COMUNA 1,2,Chile,NaN,NaN,7.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alojamiento,NaN
NaN,2,2017-01-02,retiro,RETIRO,COMUNA 1,1,NaN,NaN,Provincia de Buenos Aires,2.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion,NaN
NaN,3,2017-01-02,retiro,RETIRO,COMUNA 1,2,NaN,NaN,Córdoba,0.0,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alrededores,NaN
Francia,4,2017-01-02,retiro,RETIRO,COMUNA 1,2,Francia,NaN,NaN,4.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion,NaN
México,5,2017-01-02,retiro,RETIRO,COMUNA 1,1,México,NaN,NaN,0.0,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alrededores,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,174828,2018-07-25,hub2,SAN NICOLAS,COMUNA 1,5,NaN,NaN,La Rioja,6.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion parques,NaN
NaN,174829,2018-07-25,hub2,SAN NICOLAS,COMUNA 1,2,NaN,NaN,AMBA,NaN,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,instalaciones,NaN
NaN,174830,2018-07-25,hub2,SAN NICOLAS,COMUNA 1,3,NaN,NaN,AMBA,NaN,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,sube instalaciones,NaN


Vamos contar quantos turistas de cada continente solicitaram informações no posto de turismo.

In [31]:
data_grouped_continente = data.groupby(pais_em_continente)


In [32]:
# Dessa forma eu agrupei os indices de cada continente
data_grouped_continente.indices

{'Africa': array([  4888,   5120,   9790,  12909,  15786,  16266,  22573,  25643,
         30222,  34684,  35739,  42692,  51527,  67168,  68085,  71961,
         78209,  79125,  79856,  81137,  81467,  84198,  84423,  84574,
         85558,  85716,  85755,  88407,  88552,  88576,  88619,  88684,
         89018,  89766,  89770,  90078,  90268,  90338,  91031,  92730,
         93686,  94912,  95509,  97516,  97791,  98036,  98798,  99002,
         99323,  99376,  99871, 100067, 100278, 100516, 100677, 100883,
        100942, 101372, 101947, 102359, 102469, 102618, 102768, 104250,
        104509, 105313, 105335, 106091, 107059, 107606, 107629, 108104,
        109243, 110743, 110812, 110967, 111332, 111591, 111969, 112170,
        113373, 113568, 113607, 114316, 114384, 115087, 115335, 115908,
        116108, 116109, 116418, 117132, 117496, 117622, 118102, 121075,
        121552, 122090, 122382, 126116, 128133, 128208, 129043, 129083,
        130162, 134474, 135176, 137171, 137878, 137959

In [33]:
data_grouped_continente["pasajeros"].sum()

pais_residencia_si_extranjero
Africa        327
America    152979
Asia         6130
Europa      45316
Oceania      5932
Name: pasajeros, dtype: int64

In [34]:
data_grouped_continente["pasajeros"].mean()

pais_residencia_si_extranjero
Africa     2.056604
America    2.212662
Asia       2.123311
Europa     2.051055
Oceania    2.109531
Name: pasajeros, dtype: float64

In [35]:
data_grouped_continente["pernoctaciones"].sum()

pais_residencia_si_extranjero
Africa        562.0
America    359155.0
Asia        14045.0
Europa     161265.0
Oceania      9652.0
Name: pernoctaciones, dtype: float64

In [36]:
data_grouped_continente["pernoctaciones"].mean()

pais_residencia_si_extranjero
Africa      5.252336
America     7.636721
Asia        7.341871
Europa     10.475154
Oceania     5.021852
Name: pernoctaciones, dtype: float64

Da mesma forma, podemos indexar Series com dicionários:

In [37]:
# criamos a série
serie_passageiros = data.pasajeros

# atribuímos como índice o valor do campo pais_residencia_si_extranjero para esse registro
serie_passageiros.index = data.pais_residencia_si_extranjero

# agrupamos e somamos
serie_passageiros.groupby(pais_em_continente).sum()

pais_residencia_si_extranjero
Africa        327
America    152979
Asia         6130
Europa      45316
Oceania      5932
Name: pasajeros, dtype: int64

Agora queremos indexar um DataFrame com um objeto Series.

Vamos transformar o dicionário pais_en_continente em uma instância Series e usá-lo para indexar.

In [38]:
pais_em_continente_serie = pd.Series(pais_em_continente)
pais_em_continente_serie

Chile             America
Francia            Europa
México            America
Colombia          America
Uruguay           America
Estados Unidos    America
España             Europa
Italia             Europa
India                Asia
Ecuador           America
Brasil            America
Australia         Oceania
Bolivia           America
Reino Unido        Europa
Alemania           Europa
Israel               Asia
China                Asia
Venezuela         America
Países Bajos       Europa
Canadá            America
Suiza              Europa
Turquía            Europa
Noruega            Europa
Corea del Sur        Asia
Polonia            Europa
Perú              America
Paraguay          America
Costa Rica        America
Japón                Asia
Marruecos          Africa
Bélgica            Europa
Malasia              Asia
Rusia              Europa
Sudáfrica          Africa
Nueva Zelanda     Oceania
dtype: object

In [39]:
# também posso fazer o groupby com uma series, não necessariamente só com dicionário

data.index = data.pais_residencia_si_extranjero
data_grouped_continente_2 = data.groupby(pais_em_continente_serie)
data_grouped_continente_2["pasajeros"].sum()

Africa        327
America    152979
Asia         6130
Europa      45316
Oceania      5932
Name: pasajeros, dtype: int64

## Groupby com funções

Qualquer função que passarmos como uma chave de grupo será chamada uma vez para cada valor do índice e o resultado será o nome do grupo.

Vejamos um exemplo:

Definimos uma função que, dada uma string que representa um país, retorna o nome do contêiner daquele país:

In [40]:
def get_continente(pais):
    pais_em_continente = {
    'Chile': 'America', 'Francia': 'Europa', 'México': 'America', 'Colombia': 'America', 
    'Uruguay': 'America', 'España': 'Europa', 'Italia': 'Europa', 'Estados Unidos': 'America',
    'India': 'Asia', 'Ecuador': 'America', 'Brasil': 'America', 
    'Australia': 'Oceania', 'Bolivia': 'America', 'Reino Unido': 'Europa', 
    'Alemania': 'Europa', 'Israel': 'Asia', 'China': 'Asia', 
    'Venezuela': 'America', 'Países Bajos': 'Europa', 'Canadá': 'America', 'Suiza': 'Europa', 'Turquía': 'Europa',
    'Noruega': 'Europa', 'Corea del Sur': 'Asia', 'Polonia': 'Europa', 'Perú': 'America', 'Paraguay': 'America',
    'Costa Rica': 'America', 'Japón': 'Asia', 'Marruecos': 'Africa', 'Bélgica': 'Europa', 'Malasia': 'Asia', 
    'Rusia': 'Europa', 'Sudáfrica': 'Africa', 'Nueva Zelanda': 'Oceania'}
    if pais in pais_em_continente:
        result = pais_em_continente[pais]
    else:
        result = "desconocido"
    return result
    

In [41]:
get_continente('Chile')

'America'

Repetimos o exercício anterior agrupando com esta função.

Lembre-se que **a função que é a chave do groupby recebe como argumento o valor do índice de cada registro** quando axis = 0 (que é o valor padrão do eixo) e recebe o valor da coluna quando axis = 1.

Em todos os exercícios desta prática, agrupamos por linhas (axis = 0), mas a mesma lógica se aplica se quisermos agrupar por colunas.

In [42]:
data.index = data.pais_residencia_si_extranjero
data_grouped_func = data.groupby(get_continente, axis = 0)
data_grouped_func["pasajeros"].sum()

pais_residencia_si_extranjero
Africa            327
America        152979
Asia             6130
Europa          45316
Oceania          5932
desconocido    149525
Name: pasajeros, dtype: int64

## Que operações podemos fazer em grupos?

In [43]:
# redefinimos o índice de dados, que modificamos anteriormente 
data = data.reset_index(drop = True)
data.head()

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,pais_residencia_si_extranjero,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
0,1,2017-01-02,retiro,RETIRO,COMUNA 1,2,Chile,NaN,NaN,7.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alojamiento,NaN
1,2,2017-01-02,retiro,RETIRO,COMUNA 1,1,NaN,NaN,Provincia de Buenos Aires,2.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion,NaN
2,3,2017-01-02,retiro,RETIRO,COMUNA 1,2,NaN,NaN,Córdoba,0.0,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alrededores,NaN
3,4,2017-01-02,retiro,RETIRO,COMUNA 1,2,Francia,NaN,NaN,4.0,No especifica,NaN,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion,NaN
4,5,2017-01-02,retiro,RETIRO,COMUNA 1,1,México,NaN,NaN,0.0,No especifica,No pernocta en Buenos Aires,NaN,NaN,NaN,No especifica,No especifica,NaN,Mapas Orientacion alrededores,NaN


In [44]:
data_grouped = data.groupby('pais_residencia_si_extranjero')
data_grouped

In [45]:
data_grouped.sum()

,id,fecha,centro_atencion_turistica,barrio,comuna,pasajeros,otro_pais_residencia_si_extranjero,provincia_residencia_si_argentino,pernoctaciones,medio_transporte_llegada,alojamiento,otro_alojamiento,barrio.1,otro_barrio,primera_vez,motivo_viaje,otro_motivo_viaje,motivo_consulta,otro_motivo_consulta
pais_residencia_si_extranjero,,,,,,,,,,,,,,,,,,,
Alemania,299725486,2017-01-022017-01-022017-01-022017-01-022017-0...,florida2florida2hub2recoleta2florida2florida2f...,SAN NICOLASSAN NICOLASSAN NICOLASRECOLETASAN N...,COMUNA 1COMUNA 1COMUNA 1COMUNA 2COMUNA 1COMUNA...,7265,0,0,20309.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,CruceroCruceroCruceroCruceroCruceroBarcoPor el...,PalermoSan TelmoMontserratConstituciónSan Nico...,FloresTigre,No especificaNo especificaNo especificaNo espe...,No especificaNo especificaNo especificaNo espe...,DE PASO,bicicletassubeMapas atractivos literario museo...,TigreHerramientas para arreglar biciWifiEstadi...
Australia,218388532,2017-01-022017-01-022017-01-022017-01-022017-0...,puertomadero2florida2recoleta2recoleta2florida...,PUERTO MADEROSAN NICOLASRECOLETARECOLETASAN NI...,COMUNA 1COMUNA 1COMUNA 2COMUNA 2COMUNA 1COMUNA...,5394,0,0,8786.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,CruceroCruceroCruceroCruceroCruceroCrucero,RecoletaPalermoSan NicolásRecoletaPalermoRecol...,5,No especificaNo especificaNo especificaNo espe...,No especificaNo especificaNo especificaNo espe...,Combinaci├│n de vuelos. Pasajero en tr├ínsito,MapasBusBusBusOrientacionBusOrientacionOrienta...,Correo argentinoPiletas públicasDay saving lig...
Bolivia,74537777,2017-01-022017-01-022017-01-022017-01-022017-0...,retiropuertomadero2florida2hub2florida2puertom...,RETIROPUERTO MADEROSAN NICOLASSAN NICOLASSAN N...,COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA...,1981,0,0,7948.0,No especificaAviónNo especificaNo especificaNo...,HotelNo pernocta en Buenos AiresNo pernocta en...,0,San NicolásMontserratConstituciónSan NicolásBa...,FloresVilla crespoLiniers5Liniers,No especificaSíNo especificaNo especificaNo es...,No especificaNo especificaNo especificaNo espe...,0,Mapas Orientacion organismosMapasBus MapasMapa...,TemaikenBA playaConsulta impresión voucher vis...
Brasil,2669210899,2017-01-022017-01-022017-01-022017-01-022017-0...,hub2florida2recoleta2florida2caminitocaminitof...,SAN NICOLASSAN NICOLASRECOLETASAN NICOLASBOCAB...,COMUNA 1COMUNA 1COMUNA 2COMUNA 1COMUNA 4COMUNA...,68984,0,0,101399.0,No especificaAviónAviónNo especificaNo especif...,No pernocta en Buenos AiresHostelNo pernocta e...,CruceroCruceroCruceroestan de paso aeropuertoB...,San TelmoPalermoSan NicolásRecoletaRetiroRetir...,BoedoSan cristobalNuñezSan isidroNo sabeVilla ...,No especificaSíNo especificaNo especificaNo es...,No especificaVacaciones / Ocio / Recreación No...,escalaEscalaTr├ínsitoTr├ínsitoTr├ínsitoNegocio...,Mapas Orientacion apps visitasBusBusOrientacio...,Pronóstico del tiempopaseo en barcoSan Antonio...
Bélgica,39699810,2017-10-202017-10-212017-10-212017-10-232017-1...,hub2pzasanmartinfloridahub2caminitorecoletacam...,SAN NICOLASRETIROSAN NICOLASSAN NICOLASBOCAREC...,COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA 4COMUNA...,664,0,0,1717.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,0,RecoletaBelgranoRetiroSan NicolásRetiroSan Tel...,0,No especificaNo especificaNo especificaNo espe...,No especificaNo especificaNo especificaNo espe...,0,Mapas OrientacionBus Mapas OrientacionMapas su...,Ayuda para instalar las app de ba turismoWifi ...
Canadá,181570557,2017-01-022017-01-022017-01-022017-01-022017-0...,florida2retiroflorida2florida2florida2florida2...,SAN NICOLASRETIROSAN NICOLASSAN NICOLASSAN NIC...,COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA 1COMUNA...,4636,0,0,9249.0,No especificaNo especificaNo especificaNo espe...,No pernocta en Buenos AiresNo pernocta en Buen...,CruceroCruceroCrucerovive en argCrucero,

### Estatísticas descritivas sobre grupos

Vamos calcular o número de turistas de cada país que solicitaram informações.

Para tanto vamos selecionar a coluna `passageiros` do objeto [`DataFrameGroupBy`](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats) e adicionar esse campo nos registros que compõem cada grupo.

In [46]:
sum_por_pais = data_grouped["pasajeros"].sum()
print(type(sum_por_pais))
sum_por_pais

<class 'pandas.core.series.Series'>


pais_residencia_si_extranjero
Alemania                                     7265
Australia                                    5394
Bolivia                                      1981
Brasil                                      68984
Bélgica                                       664
Canadá                                       4636
Chile                                       20145
China                                        1858
Colombia                                    12858
Corea del Sur                                 287
Costa Rica                                    923
Ecuador                                      2895
España                                      13143
Estados Unidos                              13829
Francia                                     10718
India                                         209
Israel                                       2077
Italia                                       4726
Japón                                        1550
Malasia             

Vemos que o resultado é um objeto do tipo Series e seu índice são os valores únicos do campo que usamos como a chave do groupby.

Agora queremos ver um ranking dos países com base no número de turistas que visitam a cidade de Buenos Aires. Para isso ordenamos a série resultante do ponto anterior do maior para o menor, usando o método [`pandas.Series.sort_values()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.sort_values.html).

In [47]:
sum_por_pais_sorted = sum_por_pais.sort_values(ascending = False)
sum_por_pais_sorted

pais_residencia_si_extranjero
Brasil                                      68984
Chile                                       20145
Estados Unidos                              13829
España                                      13143
Colombia                                    12858
Francia                                     10718
Uruguay                                      9402
Alemania                                     7265
México                                       6915
Otro país de residencia si es extranjero     6092
Australia                                    5394
Reino Unido                                  5291
Italia                                       4726
Canadá                                       4636
Venezuela                                    4234
Perú                                         4159
Ecuador                                      2895
Israel                                       2077
Paraguay                                     2018
Bolivia             

Calculemos agora a média e o desvio padrão das dormidas por país.

Já calculamos groupby por país e atribuímos à variável [`data_grouped`](https://realpython.com/pandas-groupby/), selecionamos o campo pernoites e calculamos essas medidas.

- [`pandas.core.groupby.GroupBy.mean()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html#pandas.core.groupby.GroupBy.mean)

- [`pandas.core.groupby.GroupBy.std()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html#pandas.core.groupby.GroupBy.std)

Também podemos usar [`pandas.core.groupby.DataFrameGroupBy.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.describe.html) sobre os grupos.

In [48]:
data_grouped["pernoctaciones"].mean()

pais_residencia_si_extranjero
Alemania                                     8.126851
Australia                                    5.034957
Bolivia                                     12.115854
Brasil                                       5.052015
Bélgica                                      7.369099
Canadá                                       6.108983
Chile                                        5.137942
China                                        8.045283
Colombia                                    14.835038
Corea del Sur                                6.037383
Costa Rica                                   6.350000
Ecuador                                      8.503233
España                                       8.339584
Estados Unidos                               6.841423
Francia                                     18.084369
India                                       13.597015
Israel                                       8.033113
Italia                                      11.02809

In [49]:
data_grouped["pernoctaciones"].std()

pais_residencia_si_extranjero
Alemania                                    27.589412
Australia                                   11.089628
Bolivia                                     41.854586
Brasil                                      12.189930
Bélgica                                     17.463294
Canadá                                      19.441118
Chile                                       15.253540
China                                       26.056029
Colombia                                    51.700865
Corea del Sur                                5.795127
Costa Rica                                  11.601940
Ecuador                                     27.047443
España                                      27.042090
Estados Unidos                              19.617710
Francia                                     58.816535
India                                       46.803283
Israel                                      17.109797
Italia                                      35.34061

In [50]:
#Um describe específico do que estou agrupando
data_grouped["pernoctaciones"].describe()

,count,mean,std,min,25%,50%,75%,max
pais_residencia_si_extranjero,,,,,,,,
Alemania,2499.0,8.126851,27.589412,0.0,3.0,4.0,6.00,365.0
Australia,1745.0,5.034957,11.089628,0.0,3.0,4.0,5.00,365.0
Bolivia,656.0,12.115854,41.854586,0.0,3.0,4.0,7.00,365.0
Brasil,20071.0,5.052015,12.189930,0.0,3.0,4.0,5.00,365.0
Bélgica,233.0,7.369099,17.463294,0.0,3.0,5.0,6.00,175.0
Canadá,1514.0,6.108983,19.441118,0.0,3.0,4.0,5.00,365.0
Chile,6365.0,5.137942,15.253540,0.0,3.0,4.0,5.00,365.0
China,530.0,8.045283,26.056029,0.0,3.0,5.0,6.00,365.0
Colombia,3910.0,14.835038,51.700865,0.0,3.0,5.0,7.00,690.0


### Índices hierárquicos

Podemos agrupar por mais de um campo, e o resultado será uma Series ou DataFrame com um índice hierárquico definido pelos campos-chave do agrupamento.

Vamos ver como o número de turistas é distribuído por país de residência por bairro.

In [51]:
data_grouped_pais_bairro = data.groupby(["pais_residencia_si_extranjero", "barrio"])
quantidade_pasajeros_pais_bairro = data_grouped_pais_bairro["pasajeros"].sum()
quantidade_pasajeros_pais_bairro

pais_residencia_si_extranjero  barrio       
Alemania                       BOCA              731
                               PALERMO           311
                               PUERTO MADERO     650
                               RECOLETA         1631
                               RETIRO           1340
                                                ... 
Venezuela                      PALERMO           525
                               PUERTO MADERO     352
                               RECOLETA          652
                               RETIRO            644
                               SAN NICOLAS      1607
Name: pasajeros, Length: 220, dtype: int64

Vemos que o índice do objeto Result Series possui dois níveis. Se quisermos ver como foi definido:

In [52]:
quantidade_pasajeros_pais_bairro.index

MultiIndex([( 'Alemania',          'BOCA'),
            ( 'Alemania',       'PALERMO'),
            ( 'Alemania', 'PUERTO MADERO'),
            ( 'Alemania',      'RECOLETA'),
            ( 'Alemania',        'RETIRO'),
            ( 'Alemania',   'SAN NICOLAS'),
            ('Australia',          'BOCA'),
            ('Australia',       'PALERMO'),
            ('Australia', 'PUERTO MADERO'),
            ('Australia',      'RECOLETA'),
            ...
            (  'Uruguay', 'PUERTO MADERO'),
            (  'Uruguay',      'RECOLETA'),
            (  'Uruguay',        'RETIRO'),
            (  'Uruguay',   'SAN NICOLAS'),
            ('Venezuela',          'BOCA'),
            ('Venezuela',       'PALERMO'),
            ('Venezuela', 'PUERTO MADERO'),
            ('Venezuela',      'RECOLETA'),
            ('Venezuela',        'RETIRO'),
            ('Venezuela',   'SAN NICOLAS')],
           names=['pais_residencia_si_extranjero', 'barrio'], length=220)

Podemos usar o método [`pandas.Series.unstack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html) para criar um DataFrame a partir deste objeto Series.

In [53]:
# A ideia é q o "unstack" retorne as informações de uma forma tabular;
quantidade_pasajeros_pais_bairro_df = quantidade_pasajeros_pais_bairro.unstack()
print(type(quantidade_pasajeros_pais_bairro_df))
quantidade_pasajeros_pais_bairro_df.head(3)

<class 'pandas.core.frame.DataFrame'>


barrio,BARRIO,BOCA,PALERMO,PUERTO MADERO,RECOLETA,RETIRO,SAN NICOLAS,SIN IDENTIFICAR
pais_residencia_si_extranjero,,,,,,,,
Alemania,NaN,731.0,311.0,650.0,1631.0,1340.0,2602.0,NaN
Australia,NaN,553.0,193.0,296.0,1853.0,768.0,1731.0,NaN
Bolivia,NaN,375.0,222.0,165.0,255.0,401.0,563.0,NaN



<a id="section_aggregate_transform_filter"></a> 
### Aggregate, transform, filter


Uma vez construídos os grupos (como resultado da etapa `Split`), na etapa `Apply` podemos realizar as operações sobre eles, quais sejam:

* **agregação**: cálculo das estatísticas de resumo para cada grupo. Por exemplo, soma ou média

* **transformação**: cálculos específicos do grupo retornando novos objetos indexados da mesma maneira. Por exemplo, preencha os NAs dentro de um grupo com um valor calculado nesse grupo, como média, mediana, máximo, etc.

* **filtro**: descarte alguns grupos de acordo com algum cálculo no grupo que retorna Verdadeiro ou Falso. Por exemplo, descarte grupos com poucos membros.

Exemplos de operações de agregação são todos os que vimos até agora. Depois de construir um grupo com qualquer uma das alternativas que apresentamos, calculamos uma medida em cada um desses grupos.

#### Transformação - Não se preocupar muito com essa manipulação porque existem maneiras mais simples de se fazer;

Vimos que no domínio das dormidas existe uma percentagem muito elevada de nulos.

Vamos completar os valores deste campo, atribuindo a média das dormidas agrupadas por `pais_residencia_si_extranjero` e `bairro`.

Sabemos que `groupby` não configura grupos definidos por nulos, portanto aqueles registros que possuem `null` nos campos que são chave para `groupby` não serão atribuídos a nenhum grupo.

Antes de começar, vamos remover esses registros.

In [54]:
# Para ver quantos dessa máscara são não nulos. Existem formas mais simples de fazer isso
data_key_not_null_mask = np.logical_and(data.pais_residencia_si_extranjero.notnull(), data.barrio.notnull())
data_key_not_null = data.loc[data_key_not_null_mask, :]
data_key_not_null.shape

(100107, 20)

In [55]:
# se necessário, eliminamos o índice que atribuímos aos exercícios agrupados por série ou dicionário:
data_key_not_null = data_key_not_null.reset_index(drop = True)

#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
#data_key_not_null.head(3)

In [56]:
data_key_not_null_grouped_pais_bairro = data_key_not_null.groupby(["pais_residencia_si_extranjero", "barrio"])

Qual porcentagem de valores nulos estão na coluna pernoctaciones em data_key_not_null?

In [57]:
data_key_not_null["pernoctaciones"].isnull().sum() / data_key_not_null.shape[0]

0.31192623892435095

Usamos [`transform`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#transformation) para preencher os valores nulos com a média por grupo e contamos quantos valores nulos restam.

In [58]:
data_filled = data_key_not_null_grouped_pais_bairro["pernoctaciones"].transform(lambda grp: grp.fillna(grp.mean()))
data_filled

0          7.000000
1          4.000000
2          0.000000
3         18.182640
4          3.782383
            ...    
100102     4.000000
100103     6.000000
100104     5.629213
100105     2.000000
100106     4.000000
Name: pernoctaciones, Length: 100107, dtype: float64

In [59]:
data_filled.isnull().sum()

10

Vemos que existem 10 registros restantes que foram atribuídos a um grupo, mas eles ainda são nulos. **O que aconteceu?**

Vejamos o que são esses registros e quais valores eles têm nos campos "pais_residencia_si_extranjero" e "barrio".

In [60]:
data_not_filled = data_filled.loc[data_filled.isnull()]

In [61]:
data_key_not_null.loc[data_not_filled.index, [ "pais_residencia_si_extranjero", "barrio"]]

,pais_residencia_si_extranjero,barrio
53093,Malasia,RECOLETA
54936,Malasia,RECOLETA
56875,Marruecos,PALERMO
57063,Marruecos,PALERMO
57144,Malasia,RECOLETA
57492,Marruecos,PALERMO
59074,Marruecos,PALERMO
85644,Marruecos,PALERMO
95932,Estados Unidos,SIN IDENTIFICAR
95933,Brasil,SIN IDENTIFICAR


Vejamos os valores no campo "pernoctaciones" dos registros de grupo
* Malasia	RECOLETA	
* Marruecos	PALERMO	
* Estados Unidos	NÃO IDENTIFICADO
* Brasil	NÃO IDENTIFICADO

In [62]:
malasia_recoleta_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Malasia', 
                                        data_key_not_null.barrio == "RECOLETA")

data_key_not_null.loc[malasia_recoleta_mask, "pernoctaciones"]

53093   NaN
54936   NaN
57144   NaN
Name: pernoctaciones, dtype: float64

In [63]:
marruecos_palermo_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Marruecos', 
                                        data_key_not_null.barrio == "PALERMO")

data_key_not_null.loc[marruecos_palermo_mask, "pernoctaciones"]

56875   NaN
57063   NaN
57492   NaN
59074   NaN
85644   NaN
Name: pernoctaciones, dtype: float64

In [64]:
usa_sinid_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Estados Unidos', 
                                        data_key_not_null.barrio == "SIN IDENTIFICAR")

data_key_not_null.loc[usa_sinid_mask, "pernoctaciones"]

95932   NaN
Name: pernoctaciones, dtype: float64

In [65]:
brasil_sinid_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Brasil', 
                                        data_key_not_null.barrio == "SIN IDENTIFICAR")

data_key_not_null.loc[brasil_sinid_mask, "pernoctaciones"]

95933   NaN
Name: pernoctaciones, dtype: float64

Vemos que todos os registros desses grupos têm valor nulo no campo de pernoites, portanto a média por grupo também é nula e novamente temos nulo como preenchimento por grupo.

#### Filtro

O método [`filtration`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#filtration) retorna um subconjunto do objeto original.

Suponha que desejamos retornar os registros que correspondem a países com mais de 1000 visitas.

In [66]:
data_group_pais = data.groupby(data.pais_residencia_si_extranjero)
data_group_pais.size()

pais_residencia_si_extranjero
Alemania                                     3564
Australia                                    2548
Bolivia                                       902
Brasil                                      29979
Bélgica                                       304
Canadá                                       2219
Chile                                        9140
China                                         907
Colombia                                     5965
Corea del Sur                                 155
Costa Rica                                    421
Ecuador                                      1360
España                                       6249
Estados Unidos                               6826
Francia                                      5159
India                                         105
Israel                                        855
Italia                                       2373
Japón                                         801
Malasia             

In [67]:
data_paises_frequentes = data_group_pais.filter(lambda grp: grp["pasajeros"].sum() > 1000)

Tamanho antes do filtro (também estamos contando registros que têm nulos em pais_residencia_si_extranjero):

In [68]:
data.shape

(174812, 20)

Tamanho após o filtro (apenas registros não nulos no campo pais_residencia_si_extranjero):

In [69]:
data_paises_frequentes.shape

(98093, 20)

Outra maneira de calcular os tamanhos antes do filtro:

(apenas registros que não são nulos no campo pais_residencia_si_extranjero)

In [70]:
data_group_pais.size().sum()

100107

#### Apply

Podemos avaliar/aplicar funções em grupos usando [`apply`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply).

Vejamos um exemplo em que aplicamos o método [`pandas.core.groupby.DataFrameGroupBy.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.describe.html) em cada um dos grupos por país.

In [71]:
data_group_pais = data.groupby(data.pais_residencia_si_extranjero)
data_group_pais.apply(lambda grp: grp.describe())

id    pasajeros   
pais_residencia_si_extranjero                                     
Alemania                      count    3564.000000  3564.000000  \
                              mean    84098.060045     2.038440   
                              std     50825.300317     1.436243   
                              min        49.000000     1.000000   
                              25%     30424.500000     1.000000   
...                                            ...          ...   
Venezuela                     min        78.000000     1.000000   
                              25%     43255.000000     1.000000   
                              50%     83076.000000     2.000000   
                              75%    132194.000000     2.000000   
                              max    174759.000000    12.000000   

                                     pernoctaciones  
pais_residencia_si_extranjero                        
Alemania                      count     2499.000000  
                              mean         8.126851  
                              std         27.589412  
                              min          0.000000  
                              25%          3.000000  
...                                             ...  
Venezuela                     min          0.000000  
                              25%          3.000000  
                              50%          5.000000  
                              75%         15.000000  
                              max        670.000000  

[288 rows x 3 columns]

---

#### Referências

Python for Data Analysis. Wes McKinney. Cap 10

- [Group by: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

- [Grouping](https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping)